In [1]:
from fspmodel.timeIndex import FSPTimeIndexform
from fspmodel.positionIndex import positionIndex
from fspmodel.FSPasTSP import FSPasTSP
from qiskit.circuit.library import TwoLocal,EfficientSU2,QAOAAnsatz
from qiskit.algorithms.optimizers import SPSA
from qiskit.algorithms import VQE,QAOA
import numpy as np
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.problems import QuadraticProgram
from qiskit.providers.aer import QasmSimulator

In [2]:
N = 2
M = 2
pm = [[2,7,2],[1,30,1]]
T = sum(pm[m][i] for m in range(M) for i in range(N))
ops = []
# FSP AS TSP Test
for i in range(1,6):
    print("=============approach"+str(i)+"===============\n")
    fsp = FSPasTSP(M,pm,N,i)
    mdl1 = fsp.to_qubo()
    op,off = fsp.to_ising()
    print(op)
    ops.append(op)
    print("====== Position Index ==================="+"\n")
# Position Index 
fsp = positionIndex(M,pm,N)
mdl2 = fsp.to_qubo()
op2 = fsp.to_ising()
print(op2)
ops.append(op2)
print(mdl2)

=============approach1===============

-9.25 * ZIII
- 14.75 * IZII
+ 48.5 * ZZII
- 14.75 * IIZI
+ 48.5 * ZIZI
+ 14.75 * IZZI
- 9.25 * IIIZ
+ 9.25 * ZIIZ
+ 48.5 * IZIZ
+ 48.5 * IIZZ
====== Position Index ===================

=============approach2===============

-3.0 * ZIII
- 7.0 * IZII
+ 20.5 * ZZII
- 7.0 * IIZI
+ 20.5 * ZIZI
+ 7.0 * IZZI
- 3.0 * IIIZ
+ 3.0 * ZIIZ
+ 20.5 * IZIZ
+ 20.5 * IIZZ
====== Position Index ===================

=============approach3===============

-8.5 * ZIII
+ 8.5 * IZII
+ 34.5 * ZZII
+ 8.5 * IIZI
+ 34.5 * ZIZI
- 8.5 * IZZI
- 8.5 * IIIZ
+ 8.5 * ZIIZ
+ 34.5 * IZIZ
+ 34.5 * IIZZ
====== Position Index ===================

=============approach4===============

0.5 * ZIII
+ 9.25 * IZII
+ 20.0 * ZZII
+ 9.25 * IIZI
+ 20.0 * ZIZI
- 9.25 * IZZI
+ 0.5 * IIIZ
- 0.5 * ZIIZ
+ 20.0 * IZIZ
+ 20.0 * IIZZ
====== Position Index ===================

=============approach5===============

-1.5 * ZIII
- 7.0 * IZII
+ 17.5 * ZZII
- 7.0 * IIZI
+ 17.5 * ZIZI
+ 7.0 * IZZI
- 1.5 * III

In [3]:
device = QuantumInstance(QasmSimulator(method='matrix_product_state'), shots=500)
#ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz',entanglement='circular',reps=1)
#ansatz = EfficientSU2(N**2)
ansatz =QAOAAnsatz(cost_operator=ops[0], reps=9, initial_state=None, mixer_operator=None, name='QAOA')
spsa = SPSA(maxiter=1000)
vqe = VQE(ansatz, optimizer=spsa, quantum_instance=device)

In [4]:
parameters = list(ansatz.parameters)
circuits = vqe.construct_circuit(parameters, op[0])
print(circuits)
for circuit in circuits:
    print(circuit.decompose().draw())

     ┌───┐»
q_0: ┤ H ├»
     ├───┤»
q_1: ┤ H ├»
     ├───┤»
q_2: ┤ H ├»
     ├───┤»
q_3: ┤ H ├»
     └───┘»
«     ┌────────────────────────────────────────────────────────────────────────────────────────┐»
«q_0: ┤0                                                                                       ├»
«     │                                                                                        │»
«q_1: ┤1                                                                                       ├»
«     │  exp(-i ZIII + IZII + ZZII + IIZI + ZIZI + IZZI + IIIZ + ZIIZ + IZIZ + IIZZ)(1.0*γ[0]) │»
«q_2: ┤2                                                                                       ├»
«     │                                                                                        │»
«q_3: ┤3                                                                                       ├»
«     └────────────────────────────────────────────────────────────────────────────────────────┘»
«     ┌───

In [5]:
result = vqe.compute_minimum_eigenvalue(ops[0])

In [6]:
print(result.optimizer_evals,result)

None {   'aux_operator_eigenvalues': None,
    'cost_function_evals': 2000,
    'eigenstate': {   '0000': 0.17320508075688773,
                      '0001': 0.4626013402488151,
                      '0010': 0.17320508075688773,
                      '0011': 0.20493901531919198,
                      '0100': 0.161245154965971,
                      '0101': 0.161245154965971,
                      '0110': 0.12649110640673517,
                      '0111': 0.25298221281347033,
                      '1000': 0.4427188724235731,
                      '1001': 0.11832159566199232,
                      '1010': 0.21447610589527216,
                      '1011': 0.12649110640673517,
                      '1100': 0.18973665961010275,
                      '1101': 0.12649110640673517,
                      '1110': 0.27568097504180444,
                      '1111': 0.38987177379235854},
    'eigenvalue': (38.111999999999995+0j),
    'optimal_parameters': {   ParameterVectorElement(γ[3]): 4.41724877